## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [1]:
from nltk import download

download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\coolemi\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

add requred additional libraries

In [37]:
import boto3
import math
from pytictoc import TicToc
t = TicToc() #create instance of class

In [3]:
dir()

['In',
 'Out',
 'TicToc',
 '_',
 '_1',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'boto3',
 'download',
 'exit',
 'get_ipython',
 'quit',
 't']

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [4]:
from nltk.corpus import movie_reviews

In [5]:
# extract words from reviews, pair with label
t.tic()

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)
    
t.toc()

Elapsed time is 0.865447 seconds.


### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


## Score the reviews individually

In [6]:
def score_review(review):
    
    # establish an AWS session with a profile with permission to access "comprehend"
    session = boto3.Session(profile_name='ml-as-microservice')
    
    # establish a client variable to access the "comprehend" service
    comp = session.client(service_name='comprehend')
    
    # store the model's output of the review in a variable
    model_output = comp.detect_sentiment(Text = review, LanguageCode='en')
    
    # get the positive probability, and the negative probability
    pos = model_output['SentimentScore']['Positive']
    neg = model_output['SentimentScore']['Negative']
    
    # if positive is more probable, give pos.  Else, give neg.
    if pos > neg:
        function_output = 'pos'
    else:
        function_output = 'neg'
        
    return function_output

### Score each review

Now, we can use the function you defined to score each of the reviews

In [ ]:
t.tic()

results_pos = []
for review in reviews_pos:
    result = score_review(review[0:5000])
    results_pos.append(result)

t.toc()

In [ ]:
t.tic()

results_neg = []
for review in reviews_neg:
    result = score_review(review[0:5000])
    results_neg.append(result)

t.toc()

In [57]:
math.floor(1000/25)

40

## Score the reviews batch-style

In [60]:
def score_review_batch(review_list):
    # establish an AWS session with a profile with permission to access "comprehend"
    session = boto3.Session(profile_name='ml-as-microservice')
    
    # establish a client variable to access the "comprehend" service
    comp = session.client(service_name='comprehend')

    # no elements longer than 5k
    review_list[:] = (elem[0:4999] for elem in review_list)
    
    # initialize the list that will hold the results
    sentiment_list = []
    
    # batch will only receive 25 at a time.  loop through the list in sets of 25
    i = 0
    max_i = math.floor(len(review_list)/25)-1
    
    while i <= max_i:
    
        ndx_min = i*25
        ndx_max = min(ndx_min + 24, len(review_list))
        
        # print('from ', ndx_min, ' to ', ndx_max)
    
        # create a list of the model's sentiment scores for each review in the review list
        full_review_list = comp.batch_detect_sentiment(TextList = review_list[ndx_min:ndx_max], LanguageCode='en')['ResultList']

        # calc a 'pos' or 'neg' for each element of the full_review_list
        for ele in full_review_list:
            if ele['SentimentScore']['Positive'] > ele['SentimentScore']['Negative']:
                result = 'pos'
            else:
                result = 'neg'
            sentiment_list.append(result)
        
        i = i+1

    return sentiment_list

### Score each review

Now, we can use the function you defined to score each of the reviews

In [62]:
t.tic()
results_pos = score_review_batch(reviews_pos)
t.toc()

Elapsed time is 27.851296 seconds.


In [61]:
t.tic()
results_neg = score_review_batch(reviews_neg)
t.toc()

Elapsed time is 26.216157 seconds.


### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [63]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 75.52083333333334% correct
Negative reviews: 56.875% correct
Overall accuracy: 66.19791666666667% correct
